In [371]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [372]:
SEGMENT = 2

In [373]:
df = pd.read_csv(f'../preprocess/all_segments/segment{SEGMENT}.csv')

df.dropna(inplace=True)
df

,Unnamed: 0,trip_id,deviceid,direction,segment,date,start_time,end_time,run_time_in_seconds,length,weather,date_in_week,hour,time_interval,part_of_day
0,1,1.0,262.0,1.0,2.0,2021-10-01,06:42:12,06:45:42,210.0,1.2808,1,Friday,6,6-7,morning
1,16,8.0,274.0,1.0,2.0,2021-10-01,08:43:53,08:47:38,225.0,1.2808,1,Friday,8,8-9,morning
2,31,10.0,123.0,1.0,2.0,2021-10-01,09:15:09,09:18:50,221.0,1.2808,1,Friday,9,9-10,morning
3,40,15.0,262.0,1.0,2.0,2021-10-01,10:26:01,10:29:17,196.0,1.2808,1,Friday,10,10-11,morning
4,55,21.0,274.0,1.0,2.0,2021-10-01,11:56:59,11:59:59,180.0,1.2808,1,Friday,11,11-12,morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7033,171190,25364.0,1166.0,1.0,2.0,2022-11-01,16:43:40,16:47:51,251.0,1.2800,1,Tuesday,16,16-17,afternoon
7034,171205,25366.0,1358.0,1.0,2.0,2022-11-01,17:03:19,17:06:49,210.0,1.2800,1,Tuesday,17,17-18,afternoon
7035,171218,25368.0,284.0,1.0,2.0,2022-11-01,17:54:31,17:58:35,244.0,1.2800,1,Tuesday,17,17-18,afternoon
7036,171233,25369.0,513.0,1.0,2.0,2022-11-01,18:10:16,18:14:12,236.0,1.2800,1,Tuesday,18,18-19,afternoon


In [374]:

df = pd.get_dummies(df, columns=['date_in_week'])
def label_encoding(x):
    help = {
        'Monday':2,
        'Tuesday':3,
        'Wednesday': 4,
        'Thursday': 5,
        'Friday': 6,
        'Saturday': 7,
        'Sunday': 1,
        'morning': 0,
        'afternoon': 1,
        'evening': 2
    }
    return help[x]


# df['date_in_week'] = df['date_in_week'].apply(label_encoding)
df['part_of_day'] = df['part_of_day'].apply(label_encoding)

df

,Unnamed: 0,trip_id,deviceid,direction,segment,date,start_time,end_time,run_time_in_seconds,length,...,hour,time_interval,part_of_day,date_in_week_Friday,date_in_week_Monday,date_in_week_Saturday,date_in_week_Sunday,date_in_week_Thursday,date_in_week_Tuesday,date_in_week_Wednesday
0,1,1.0,262.0,1.0,2.0,2021-10-01,06:42:12,06:45:42,210.0,1.2808,...,6,6-7,0,True,False,False,False,False,False,False
1,16,8.0,274.0,1.0,2.0,2021-10-01,08:43:53,08:47:38,225.0,1.2808,...,8,8-9,0,True,False,False,False,False,False,False
2,31,10.0,123.0,1.0,2.0,2021-10-01,09:15:09,09:18:50,221.0,1.2808,...,9,9-10,0,True,False,False,False,False,False,False
3,40,15.0,262.0,1.0,2.0,2021-10-01,10:26:01,10:29:17,196.0,1.2808,...,10,10-11,0,True,False,False,False,False,False,False
4,55,21.0,274.0,1.0,2.0,2021-10-01,11:56:59,11:59:59,180.0,1.2808,...,11,11-12,0,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7033,171190,25364.0,1166.0,1.0,2.0,2022-11-01,16:43:40,16:47:51,251.0,1.2800,...,16,16-17,1,False,False,False,False,False,True,False
7034,171205,25366.0,1358.0,1.0,2.0,2022-11-01,17:03:19,17:06:49,210.0,1.2800,...,17,17-18,1,False,False,False,False,False,True,False
7035,171218,25368.0,284.0,1.0,2.0,2022-11-01,17:54:31,17:58:35,244.0,1.2800,...,17,17-18,1,False,False,False,False,False,True,False
7036,171233,25369.0,513.0,1.0,2.0,2022-11-01,18:10:16,18:14:12,236.0,1.2800,...,18,18-19,1,False,False,False,False,False,True,False


In [375]:
# Extract the training and test data
data = df.values
X = data[:,[1,2,3,10,13,14,15,16,17,18,19,20]]
# X = data[:,[1,2,3,10,11,14]]
y = data[:,8]
X

array([[1.0, 262.0, 1.0, ..., False, False, False],
       [8.0, 274.0, 1.0, ..., False, False, False],
       [10.0, 123.0, 1.0, ..., False, False, False],
       ...,
       [25368.0, 284.0, 1.0, ..., False, True, False],
       [25369.0, 513.0, 1.0, ..., False, True, False],
       [25370.0, 121.0, 1.0, ..., False, True, False]], dtype=object)

In [376]:
# Tính Z-score cho mỗi cột trong dữ liệu X
z_scores = np.abs((X - X.mean()) / X.std())

# Định một ngưỡng Z-score tùy ý (ví dụ: 3.0) để xác định outlier
threshold = 3.0

# Loại bỏ các dòng chứa outlier
X_no_outliers = X[(z_scores < threshold).all(axis=1)]
y_no_outliers = y[(z_scores < threshold).all(axis=1)]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train

array([[3839.0, 121.0, 1.0, ..., False, False, False],
       [2802.0, 117.0, 1.0, ..., False, True, False],
       [7042.0, 1143.0, 1.0, ..., False, True, False],
       ...,
       [21063.0, 128.0, 1.0, ..., False, False, False],
       [21890.0, 262.0, 1.0, ..., False, False, False],
       [2096.0, 116.0, 1.0, ..., False, False, False]], dtype=object)

In [377]:
# # Scale the data to be between 0 and 1
# #scaler = StandardScaler()
# scaler = MinMaxScaler()
# scaler.fit(X_train[:,[0,1,2,3,5]])
# X_train = scaler.transform(X_train[:,[0,1,2,3,5]])
# X_test = scaler.transform(X_test[:,[0,1,2,3,5]])

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train


array([[1.51287004e-01, 3.11915159e-03, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.10410343e-01, 6.23830318e-04, 0.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [2.77543459e-01, 6.40673737e-01, 0.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       ...,
       [8.30225866e-01, 7.48596382e-03, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.62824707e-01, 9.10792265e-02, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.25811029e-02, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [378]:
# Khởi tạo mô hình SVM với kernel tuyến tính và siêu tham số C mặc định
svm_regressor = SVR(kernel='rbf', C=1000, epsilon=18)
# Khởi tạo mô hình k-NN với số lân cận k (ví dụ: k=50)
knn_regressor = KNeighborsRegressor(n_neighbors=23)
# Khởi tạo mô hình RandomForestRegressor với số cây (n_estimators) và các tham số khác
rf_regressor = RandomForestRegressor(n_estimators=10, random_state=26)


# Huấn luyện mô hình trên tập huấn luyện
rf_regressor.fit(X_train, y_train)
svm_regressor.fit(X_train, y_train)
knn_regressor.fit(X_train, y_train)



# Dự đoán thời gian đến trên tập kiểm tra
y_pred_rf = rf_regressor.predict(X_test)
y_pred_svm = svm_regressor.predict(X_test)
y_pred_knn = knn_regressor.predict(X_test)


# Đánh giá mô hình
mae_svm = mean_absolute_error(y_test, y_pred_svm)
mae_knn = mean_absolute_error(y_test, y_pred_knn)
mae_rf = mean_absolute_error(y_test, y_pred_rf)

mse_svm = mean_squared_error(y_test, y_pred_svm)
mse_knn = mean_squared_error(y_test, y_pred_knn)
mse_rf = mean_squared_error(y_test, y_pred_rf)

r2_svm = r2_score(y_test, y_pred_svm)
r2_knn = r2_score(y_test, y_pred_knn)
r2_rf = r2_score(y_test, y_pred_rf)


print(f"Mean Absolute Error (SVM): {mae_svm}")
print(f"Mean Squared Error (SVM): {mse_svm}")
print(f"Root Mean Squared Error (SVM): {math.sqrt(mse_svm)}\n")

print(f"Mean Absolute Error (kNN): {mae_knn}")
print(f"Mean Squared Error (kNN): {mse_knn}")
print(f"Root Mean Squared Error (kNN): {math.sqrt(mse_knn)}\n")


print(f"Mean Absolute Error (RF): {mae_rf}")
print(f"Mean Squared Error (RF): {mse_rf}")
print(f"Root Mean Squared Error (RF): {math.sqrt(mse_rf)}")

Mean Absolute Error (SVM): 44.63523360832177
Mean Squared Error (SVM): 3772.9983868188156
Root Mean Squared Error (SVM): 61.424737580382185

Mean Absolute Error (kNN): 46.410789278656125
Mean Squared Error (kNN): 3791.113699465114
Root Mean Squared Error (kNN): 61.57202042701794

Mean Absolute Error (RF): 43.04680397727273
Mean Squared Error (RF): 3356.013146306818
Root Mean Squared Error (RF): 57.93110689695837


In [379]:
# # Try to get the lowest RMSE by using different Epsilon value
epsilons = np.arange(10,20)
scores = []
for e in epsilons:
    svm_regressor.set_params(kernel='rbf', epsilon=e)
    svm_regressor.fit(X_train, y_train)
    scores.append(math.sqrt(mean_squared_error(y_test, svm_regressor.predict(X_test))))
plt.plot(epsilons, scores)
plt.title("Epsilon effect")
plt.xlabel("epsilon")
plt.ylabel("RMSE")
plt.show()

KeyboardInterrupt: 

In [ ]:
# # Try to get the lowest RMSE by using different Epsilon value
C = np.array([   1,  10, 100, 200, 300])
scores = []
for c in C:
    svm_regressor.set_params(kernel='rbf', C=c)
    svm_regressor.fit(X_train, y_train)
    scores.append(math.sqrt(mean_squared_error(y_test, svm_regressor.predict(X_test))))
plt.plot(C, scores)
plt.title("Epsilon effect")
plt.xlabel("C")
plt.ylabel("RMSE")
plt.show()